In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import urllib
import requests
import re
import nltk
import datetime
import re

import pandas as pd

In [2]:
scr = []

In [3]:
for i in range(1, 4):
    request = requests.get('http://www.imdb.com/title/tt0060028/episodes?season={}'.format(i))
    scr.append(request)

In [4]:
scr

[<Response [200]>, <Response [200]>, <Response [200]>]

In [5]:
many_soups = []
for ep in scr:
    many_soups.append(BeautifulSoup(ep.content, "lxml"))

In [6]:
ep_name_list = []
number_list = []
date_list = []
rating_list = []
season_number_list = []

for season in many_soups:
    # Gets season number
    season_tag = season.find('h3',{'id': 'episode_top'})
    season_tag_text = season_tag.text.replace(u'\xa0', u' ')
    pattern = re.compile(r'[0-9]')
    season_number = re.search(pattern, season_tag_text)
    season_number = season_number.group(0) 
    
    for ep in season.find_all('div', {'class': 'info'}):
        # append ep_name
        ep_name = ep.find('a', {'itemprop': 'name'}).text
        ep_name_list.append(ep_name)
        
        # append ep number
        number = ep.find('meta')
        number = number.attrs['content']
        number = int(number)
        number_list.append(number)

        # append airdate
        date = ep.find('div', {'class': 'airdate'}).contents[0].strip()
        date_list.append(date)

        # append rating
        rating = ep.find('span', {'class': 'ipl-rating-star__rating'}).text
        rating = float(rating)
        rating_list.append(rating)
        
        season_number_list.append(season_number)

In [7]:
imdb_data = pd.DataFrame(
    {'season': season_number_list,
     'number': number_list,
    'ep_name': ep_name_list,
    'rating': rating_list,
     'airdate': date_list,
    })

In [9]:
imdb_data.describe()

,number,rating
count,80.000000,80.000000
mean,13.575000,7.497500
std,7.886497,0.832105
min,0.000000,5.200000
25%,7.000000,7.000000
50%,13.500000,7.500000
75%,20.000000,8.000000
max,29.000000,9.300000


In [10]:
imdb_data['index'] = list(range(1, 81))

In [12]:
imdb_data.head()

,airdate,ep_name,number,rating,season,index
0,27 Nov. 1988,The Cage,0,7.7,1,1
1,8 Sep. 1966,The Man Trap,1,7.3,1,2
2,15 Sep. 1966,Charlie X,2,7.2,1,3
3,22 Sep. 1966,Where No Man Has Gone Before,3,7.8,1,4
4,29 Sep. 1966,The Naked Time,4,8.0,1,5


In [13]:
imdb_data.tail()

,airdate,ep_name,number,rating,season,index
75,21 Feb. 1969,The Way to Eden,20,5.5,3,76
76,28 Feb. 1969,The Cloud Minders,21,7.0,3,77
77,7 Mar. 1969,The Savage Curtain,22,6.8,3,78
78,14 Mar. 1969,All Our Yesterdays,23,8.3,3,79
79,3 Jun. 1969,Turnabout Intruder,24,6.9,3,80


In [14]:
imdb_data.to_csv('tos_imdb_data.csv', index=False)

In [15]:
df_test = pd.read_csv('./tos_imdb_data.csv')

In [16]:
df_test.head()

,airdate,ep_name,number,rating,season,index
0,27 Nov. 1988,The Cage,0,7.7,1,1
1,8 Sep. 1966,The Man Trap,1,7.3,1,2
2,15 Sep. 1966,Charlie X,2,7.2,1,3
3,22 Sep. 1966,Where No Man Has Gone Before,3,7.8,1,4
4,29 Sep. 1966,The Naked Time,4,8.0,1,5
